# Toronto Segmentation - Week 3 Assigment - Part 2

## Ibrahim Albatly

In [42]:
# import libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests
print('Libraries imported.')

Libraries imported.


In [43]:
# Reading Table from Wikipedia HTML page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tor_post = pd.read_html(url)

#Figuring our how many tables on that HTML in Wikipediator
print ('Number of tables on Toronto Wekepedia HTML is ',len(tor_post))

# Display each table to check which one is the right table
tor_post[0]

# Copying table on HTML page into a dataframe
tor_df = tor_post[0]

#Exploring Dataframe 
print(tor_df.head())

#print out dataframe columns
print(tor_df.columns)

Number of tables on Toronto Wekepedia HTML is  3
  Postal Code           Borough               Neighborhood
0         M1A      Not assigned                        NaN
1         M2A      Not assigned                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
Index(['Postal Code', 'Borough', 'Neighborhood'], dtype='object')


In [44]:
# Only process the cells that have an assigned borough. drop rows with a borough that is Not assigned.
tor_df = tor_df[tor_df['Borough']!='Not assigned']

# To ensure that all rows with Not Assigned Borough were droped
tor_df[tor_df['Borough']!='Not assigned']['Borough'].value_counts()
tor_df['Borough'].isnull().sum()

#Rename Postal Co
tor_df = tor_df.reset_index(drop=True)
# Renaming Postal Code to Postal_Code

tor_df = tor_df.rename(columns={'Postal Code': 'Postal_Code'})

tor_df.columns

Index(['Postal_Code', 'Borough', 'Neighborhood'], dtype='object')

In [45]:
# Combining rows with similar postal code

tor_df = tor_df.groupby(['Postal_Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
tor_df.columns = ['Postal_Code','Borough','Neighborhood']
tor_df.head()


,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [46]:
#Replace Non Assigned Neighborhood by Borough Name

tor_df.loc[tor_df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = tor_df['Borough']


In [47]:
#Print out shape of dataframe 
print(tor_df.shape)
tor_df.head()

(103, 3)


,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [48]:
tor_geo = pd.read_csv('Geospatial_Coordinates.csv')
tor_geo = tor_geo.rename(columns={'Postal Code': 'Postal_Code'})

tor_df = pd.merge(tor_df, tor_geo, on='Postal_Code')

tor_df[tor_df['Postal_Code']=='M5A']

tor_df.head(12)

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
